In [2]:
import pandas as pd

recipe_df = pd.read_csv('input_data/recipes_data.csv')
recipe_df

,recipe_urls,recipe_name,serves,cooking_time,difficulty,diet,ingredients
0,https://www.jamieoliver.com/recipes/vegetable-...,Asparagus stir-fry,4,NaN,Not too tricky,"['glutenfree', 'dairyfree', 'vegetarian', 'veg...","['350 g firm tofu', '2 teaspoons groundnut oil..."
1,https://www.jamieoliver.com/recipes/fish-recip...,Sweet potato fishcakes,4,1 hour 10 minutes,Not too tricky,NaN,"['500 g potatoes', '500 g sweet potatoes', '2 ..."
2,https://www.jamieoliver.com/recipes/chicken-re...,Spring chicken stew,4,1 hour 10 minutes,Not too tricky,NaN,"['olive oil', '2 rashers of higher-welfare smo..."
3,https://www.jamieoliver.com/recipes/chicken-re...,Chicken goujons,4,23 minutes,Not too tricky,NaN,"['6 wholemeal pittas', '4 x 120 g free-range s..."
4,https://www.jamieoliver.com/recipes/vegetable-...,Sweet & sour stir-fry,2,21 minutes,Super easy,"['vegetarian', 'dairyfree', 'vegan', 'glutenfr...","['100 g fine rice noodles', '1 x 227 g tin of ..."
...,...,...,...,...,...,...,...
1703,https://www.jamieoliver.com/recipes/vegetables...,Braised greens,4,15 minutes,Super easy,"['dairyfree', 'glutenfree', 'vegan', 'vegetari...","['2 large handfuls of silverbeet', '2 large ha..."
1704,https://www.jamieoliver.com/recipes/cheese-rec...,Beautiful baked Camembert,4,30 minutes,Super easy,['vegetarian'],"['250 g Camembert', '1 clove garlic', 'a few t..."
1705,https://www.jamieoliver.com/recipes/vegetables...,Baked new potatoes with sea salt & rosemary,6,40 minutes,Super easy,"['dairyfree', 'glutenfree', 'vegan', 'vegetari...","['1 kg Jersey Royal potatoes', '2 sprigs of fr..."
1706,https://www.jamieoliver.com/recipes/fish-recip...,Baby Yorkshire puds (creamy smoked trout & hor...,6,40 minutes plus chilling time,Not too tricky,NaN,"['For the creamy smoked fish', '125 g cream ch..."


In [3]:
recipe_df['difficulty'].unique()

array(['Not too tricky', 'Super easy', 'Showing off'], dtype=object)

In [2]:
import pandas as pd 
import nltk
import string
import ast
import re
import unidecode
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from collections import Counter
import config

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pranya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pranya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def ingredient_parser(ingreds):
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', 'tbl.', 'tb', 'tbsp.', 'fluid ounce', 'fl oz', 'gill', 'cup', 'c', 'pint', 'p', 'pt', 'fl pt', 'quart', 'q', 'qt', 'fl qt', 'gallon', 'g', 'gal', 'ml', 'milliliter', 'millilitre', 'cc', 'mL', 'l', 'liter', 'litre', 'L', 'dl', 'deciliter', 'decilitre', 'dL', 'bulb', 'level', 'heaped', 'rounded', 'whole', 'pinch', 'medium', 'slice', 'pound', 'lb', '#', 'ounce', 'oz', 'mg', 'milligram', 'milligramme', 'g', 'gram', 'gramme', 'kg', 'kilogram', 'kilogramme', 'x', 'of', 'mm', 'millimetre', 'millimeter', 'cm', 'centimeter', 'centimetre', 'm', 'meter', 'metre', 'inch', 'in', 'milli', 'centi', 'deci', 'hecto', 'kilo']
#     words_to_remove = ['fresh', 'oil', 'a', 'red', 'bunch', 'and', 'clove', 'or', 'leaf', 'chilli', 'large', 'extra', 'sprig', 'ground', 'handful', 'free', 'small', 'pepper', 'virgin', 'range', 'from', 'dried', 'sustainable', 'black', 'peeled', 'higher', 'welfare', 'seed', 'for', 'finely', 'freshly', 'sea', 'quality', 'white', 'ripe', 'few', 'piece', 'source', 'to', 'organic', 'flat', 'smoked', 'ginger', 'sliced', 'green', 'picked', 'the', 'stick', 'plain', 'plus', 'mixed', 'mint', 'bay', 'basil', 'your', 'cumin', 'optional', 'fennel', 'serve', 'mustard', 'unsalted', 'baby', 'paprika', 'fat', 'ask', 'natural', 'skin', 'roughly', 'into', 'such', 'cut', 'good', 'brown', 'grated', 'trimmed', 'oregano', 'powder', 'yellow', 'dusting', 'knob', 'frozen', 'on', 'deseeded', 'low', 'runny', 'balsamic', 'cooked', 'streaky', 'nutmeg', 'sage', 'rasher', 'zest', 'pin', 'groundnut', 'breadcrumb', 'turmeric', 'halved', 'grating', 'stalk', 'light', 'tinned', 'dry', 'soft', 'rocket', 'bone', 'colour', 'washed', 'skinless', 'leftover', 'splash', 'removed', 'dijon', 'thick', 'big', 'hot', 'drained', 'sized', 'chestnut', 'watercress', 'fishmonger', 'english', 'dill', 'caper', 'raw', 'worcestershire', 'flake', 'cider', 'cayenne', 'tbsp', 'leg', 'pine', 'wild', 'if', 'fine', 'herb', 'almond', 'shoulder', 'cube', 'dressing', 'with', 'chunk', 'spice', 'thumb', 'garam', 'new', 'little', 'punnet', 'peppercorn', 'shelled', 'saffron', 'other''chopped', 'salt', 'olive', 'taste', 'can', 'sauce', 'water', 'diced', 'package', 'italian', 'shredded', 'divided', 'parsley', 'vinegar', 'all', 'purpose', 'crushed', 'juice', 'more', 'coriander', 'bell', 'needed', 'thinly', 'boneless', 'half', 'thyme', 'cubed', 'cinnamon', 'cilantro', 'jar', 'seasoning', 'rosemary', 'extract', 'sweet', 'baking', 'beaten', 'heavy', 'seeded', 'tin', 'vanilla', 'uncooked', 'crumb', 'style', 'thin', 'nut', 'coarsely', 'spring', 'chili', 'cornstarch', 'strip', 'cardamom', 'rinsed', 'honey', 'cherry', 'root', 'quartered', 'head', 'softened', 'container', 'crumbled', 'frying', 'lean', 'cooking', 'roasted', 'warm', 'whipping', 'thawed', 'corn', 'pitted', 'sun', 'kosher', 'bite', 'toasted', 'lasagna', 'split', 'melted', 'degree', 'lengthwise', 'romano', 'packed', 'pod', 'anchovy', 'rom', 'prepared', 'juiced', 'fluid', 'floret', 'room', 'active', 'seasoned', 'mix', 'deveined', 'lightly', 'anise', 'thai', 'size', 'unsweetened', 'torn', 'wedge', 'sour', 'basmati', 'marinara', 'dark', 'temperature', 'garnish', 'bouillon', 'loaf', 'shell', 'reggiano', 'canola', 'parmigiano', 'round', 'canned', 'ghee', 'crust', 'long', 'broken', 'ketchup', 'bulk', 'cleaned', 'condensed', 'sherry', 'provolone', 'cold', 'soda', 'cottage', 'spray', 'tamarind', 'pecorino', 'shortening', 'part', 'bottle', 'sodium', 'cocoa', 'grain', 'french', 'roast', 'stem', 'link', 'firm', 'asafoetida', 'mild', 'dash', 'boiling']
    words_to_remove = ['olive', 'fresh', 'oil', 'a', 'red', 'bunch', 'and', 'clove', 'or', 'groundnut', 'leaf', 'chilli', 'large', 'extra', 'sprig', 'ground', 'handful', 'free', 'small', 'pepper', 'virgin', 'range', 'from', 'dried', 'sustainable', 'black', 'peeled', 'higher', 'welfare', 'seed', 'for', 'finely', 'freshly', 'sea', 'quality', 'white', 'ripe', 'few', 'piece', 'source', 'to', 'organic', 'flat', 'smoked', 'sliced', 'green', 'picked', 'the', 'stick', 'plain', 'plus', 'mixed', 'your', 'optional', 'serve', 'mustard', 'unsalted', 'baby', 'paprika', 'fat', 'ask', 'natural', 'skin', 'roughly', 'into', 'such', 'cut', 'good', 'brown', 'grated', 'trimmed', 'powder', 'yellow', 'dusting', 'knob', 'frozen', 'on', 'deseeded', 'low', 'runny', 'balsamic', 'cooked', 'streaky', 'rasher', 'zest', 'pin', 'breadcrumb', 'halved', 'grating', 'stalk', 'light', 'tinned', 'dry', 'soft', 'rocket', 'bone', 'colour', 'washed', 'skinless', 'leftover', 'splash', 'removed', 'dijon', 'thick', 'big', 'hot', 'drained', 'sized', 'chestnut', 'watercress', 'fishmonger', 'english', 'raw', 'flake', 'cider', 'cayenne', 'tbsp', 'leg', 'pine', 'wild', 'if', 'fine', 'herb', 'shoulder', 'cube', 'dressing', 'with', 'chunk', 'spice', 'thumb', 'garam', 'new', 'little', 'punnet', 'peppercorn', 'shelled', 'saffron', 'other', 'chopped', 'salt', 'taste', 'can', 'sauce', 'water', 'diced', 'package', 'italian', 'shredded', 'divided', 'all', 'purpose', 'crushed', 'juice', 'more', 'bell', 'needed', 'thinly', 'boneless', 'half', 'cubed', 'jar', 'seasoning', 'extract', 'sweet', 'baking', 'beaten', 'heavy', 'seeded', 'tin', 'uncooked', 'crumb', 'style', 'thin', 'coarsely', 'spring', 'strip', 'rinsed', 'root', 'quartered', 'head', 'softened', 'container', 'crumbled', 'frying', 'lean', 'cooking', 'roasted', 'warm', 'whipping', 'thawed', 'pitted', 'sun', 'kosher', 'bite', 'toasted', 'split', 'melted', 'degree', 'lengthwise', 'romano', 'packed', 'pod', 'rom', 'prepared', 'juiced', 'fluid', 'floret', 'room', 'active', 'seasoned', 'mix', 'deveined', 'lightly', 'thai', 'size', 'unsweetened', 'torn', 'wedge', 'sour', 'basmati', 'dark', 'temperature', 'garnish', 'loaf', 'shell', 'reggiano', 'canola', 'parmigiano', 'round', 'canned', 'ghee', 'crust', 'long', 'broken', 'bulk', 'cleaned', 'provolone', 'cold', 'spray', 'part', 'bottle', 'sodium', 'grain', 'french', 'roast', 'stem', 'link', 'firm', 'mild', 'dash', 'boiling']
    # The ingredient list is now a string so we need to turn it back into a list. We use ast.literal_eval
    if isinstance(ingreds, list):
        ingredients = ingreds
    else:
        ingredients = ast.literal_eval(ingreds)
    # We first get rid of all the punctuation. We make use of str.maketrans. It takes three input 
    # arguments 'x', 'y', 'z'. 'x' and 'y' must be equal-length strings and characters in 'x'
    # are replaced by characters in 'y'. 'z' is a string (string.punctuation here) where each character
    #  in the string is mapped to None. 
    translator = str.maketrans('', '', string.punctuation)
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for i in ingredients:
        i.translate(translator)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', i)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items] #''.join((c for c in unicodedata.normalize('NFD', items) if unicodedata.category(c) != 'Mn'))
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # get rid of stop words
        stop_words = set(stopwords.words('english'))
        items = [word for word in items if word not in stop_words]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measures]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
            ingred_list.append(' '.join(items)) 
    ingred_list = " ".join(ingred_list)
    return ingred_list

In [11]:
vocabulary = nltk.FreqDist()
for ingredients in recipe_df['ingredients']:
    ingredients = ingredients.split()
    vocabulary.update(ingredients)
for word, frequency in vocabulary.most_common(200):
    print(f'{word};{frequency}')

'1;4816
g;4061
,;3400
of;3202
'2;2353
fresh;1910
oil',;1225
tablespoons;1031
teaspoon;954
'½;947
'4;902
large;862
ml;858
or;844
red;838
bunch;739
and;728
a;668
'3;632
tablespoon;613
'olive;587
ground;574
free-range;549
x;532
garlic',;527
olive;525
'100;516
small;488
cloves;465
sugar',;458
sprigs;442
extra;430
for;430
'200;427
virgin;390
handful;386
'50;375
dried;373
plus;366
['1;362
black;319
'6;311
to;309
unsalted;306
from;303
ripe;302
tomatoes',;302
seeds',;299
onions',;295
flour',;294
lemon',;292
heaped;291
sustainable;287
finely;285
'extra;284
pepper',;284
caster;282
'a;281
higher-welfare;279
'150;278
white;269
vinegar',;268
'250;268
butter',;266
teaspoons;264
plain;264
chopped',;262
eggs',;257
quality;255
kg;255
salt',;254
few;244
'300;238
milk',;236
'500;234
cheese',;230
['2;228
chilli',;226
leaves;226
onion',;214
peeled;209
400;208
'sea;205
wine;204
smoked;201
leaves',;201
chicken;199
'freshly;196
flat-leaf;193
the;192
whole;189
sauce',;186
mixed;186
organic;185
'400;182
butter;

In [12]:
fdist = nltk.FreqDist(ingredients)
common_words = []
for word, _ in fdist.most_common(250):
    common_words.append(word)
print(common_words)

['of', "'1", "'4", 'large', 'ASPARAGUS', '&', "oil',", "'2", 'handfuls', 'fresh', "'500", 'g', "asparagus',", "butter',", 'handful', 'lemon', "leaves',", "'olive", 'knob', 'small', "'8", 'squeeze', "juice',", "'extra", 'virgin', 'olive', ',', "['STEAMED", 'QUICK', 'TOMATO', "SAUCE',", 'sprigs', "rosemary',", 'rashers', 'higher-welfare', 'smoked', 'streaky', "bacon',", 'ripe', "tomatoes',", "'WORCESTERSHIRE", 'MUSHROOMS', 'ON', "TOAST',", 'cloves', "garlic',", 'slices', "bread',", "mushrooms',", 'flat-leaf', "parsley',", 'tablespoons', 'Worcestershire', "sauce',", "'1-2", 'knobs', "'LEMON", 'BUTTERED', 'GRILLED', 'SHAVED', 'LANCASHIRE', "CHEESE',", 'salad', "'crumbly", 'British', 'cheese', 'to', "serve',", "'MINTED", 'RAW', 'SALAD', 'WITH', 'BABY', 'SPINACH', 'FRESH', 'GARDEN', "PEAS',", 'baby', 'spinach', 'spring', "peas',", 'mint', 'asparagus', "spears',", 'juice', "of',", '"1', 'soft', "goat's", 'cheese"]']


In [13]:
recipe_df['ingredients_parsed'] = recipe_df['ingredients'].apply(lambda x: ingredient_parser(x))
recipe_df

,recipe_urls,recipe_name,serves,cooking_time,difficulty,diet,ingredients,ingredients_parsed
0,https://www.jamieoliver.com/recipes/vegetable-...,Asparagus stir-fry,4,NaN,Not too tricky,"['glutenfree', 'dairyfree', 'vegetarian', 'veg...","['350 g firm tofu', '2 teaspoons groundnut oil...",tofu sesame cornflour soy wine vinegar garlic ...
1,https://www.jamieoliver.com/recipes/fish-recip...,Sweet potato fishcakes,4,1 hour 10 minutes,Not too tricky,NaN,"['500 g potatoes', '500 g sweet potatoes', '2 ...",potato potato chipotle tabasco fish fillet bon...
2,https://www.jamieoliver.com/recipes/chicken-re...,Spring chicken stew,4,1 hour 10 minutes,Not too tricky,NaN,"['olive oil', '2 rashers of higher-welfare smo...",bacon rosemary onion carrot potato pearl barle...
3,https://www.jamieoliver.com/recipes/chicken-re...,Chicken goujons,4,23 minutes,Not too tricky,NaN,"['6 wholemeal pittas', '4 x 120 g free-range s...",wholemeal pitta chicken breast basil egg garli...
4,https://www.jamieoliver.com/recipes/vegetable-...,Sweet & sour stir-fry,2,21 minutes,Super easy,"['vegetarian', 'dairyfree', 'vegan', 'glutenfr...","['100 g fine rice noodles', '1 x 227 g tin of ...",rice noodle pineapple cornflour vinegar soy se...
...,...,...,...,...,...,...,...,...
1703,https://www.jamieoliver.com/recipes/vegetables...,Braised greens,4,15 minutes,Super easy,"['dairyfree', 'glutenfree', 'vegan', 'vegetari...","['2 large handfuls of silverbeet', '2 large ha...",silverbeet chicory garlic lemon
1704,https://www.jamieoliver.com/recipes/cheese-rec...,Beautiful baked Camembert,4,30 minutes,Super easy,['vegetarian'],"['250 g Camembert', '1 clove garlic', 'a few t...",camembert garlic tip rosemary bread stale rose...
1705,https://www.jamieoliver.com/recipes/vegetables...,Baked new potatoes with sea salt & rosemary,6,40 minutes,Super easy,"['dairyfree', 'glutenfree', 'vegan', 'vegetari...","['1 kg Jersey Royal potatoes', '2 sprigs of fr...",jersey royal potato rosemary
1706,https://www.jamieoliver.com/recipes/fish-recip...,Baby Yorkshire puds (creamy smoked trout & hor...,6,40 minutes plus chilling time,Not too tricky,NaN,"['For the creamy smoked fish', '125 g cream ch...",creamy fish cream cheese jarred horseradish le...


In [14]:
vocabulary = nltk.FreqDist()
for ingredients in recipe_df['ingredients_parsed']:
    ingredients = ingredients.split()
    vocabulary.update(ingredients)
for word, frequency in vocabulary.most_common(200):
    print(f'{word};{frequency}')

garlic;677
onion;644
lemon;629
sugar;611
butter;500
tomato;493
egg;492
flour;458
cheese;414
vinegar;331
wine;309
milk;297
coriander;293
caster;288
chicken;242
lime;236
yoghurt;233
ginger;225
parsley;222
orange;221
mint;220
potato;213
thyme;205
cream;202
vanilla;196
vegetable;191
stock;190
rosemary;181
carrot;177
cinnamon;173
bean;164
rice;158
parmesan;153
honey;152
golden;148
basil;146
bay;140
fillet;138
fennel;134
cherry;132
bread;130
cumin;130
plum;128
chocolate;125
almond;123
icing;119
celery;118
apple;104
greasing;104
pork;103
coconut;99
sesame;93
nut;93
bacon;90
nutmeg;89
syrup;89
spinach;88
pea;84
paste;84
self;83
raising;83
mushroom;82
oregano;79
cocoa;79
ice;78
pastry;76
soy;75
beef;75
double;68
salad;67
gluten;67
lettuce;65
semi;63
sage;63
shallot;62
cheddar;61
skimmed;60
leek;60
squash;58
lamb;58
cabbage;58
turmeric;58
courgette;55
avocado;55
curry;55
salmon;55
creme;55
anchovy;54
fish;53
yolk;53
pomegranate;53
banana;52
oat;52
feta;51
fraiche;51
wholemeal;50
cucumber;49
clem

In [15]:
fdist = nltk.FreqDist(ingredients)
common_words = []
for word, _ in fdist.most_common(250):
    common_words.append(word)
print(common_words)

['asparagus', 'lemon', 'butter', 'cheese', 'tomato', 'worcestershire', 'mushroom', 'squeeze', 'salad', 'spinach', 'pea', 'steamed', 'quick', 'rosemary', 'bacon', 'toast', 'garlic', 'bread', 'parsley', 'buttered', 'grilled', 'shaved', 'lancashire', 'crumbly', 'british', 'minted', 'garden', 'mint', 'spear']


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle 

# load in parsed recipe dataset 
recipe_df = pd.read_csv('input_data/parsed_ingreds_recipes.csv')
recipe_df['ingredients_parsed'] = recipe_df.ingredients_parsed.values.astype('U')

# TF-IDF feature extractor 
tfidf = TfidfVectorizer()
tfidf.fit(recipe_df['ingredients_parsed'])
tfidf_recipe = tfidf.transform(recipe_df['ingredients_parsed'])

print('Model',tfidf)
print('Encoding',tfidf_recipe)

Model TfidfVectorizer()
Encoding   (0, 1577)	0.1928619558592839
  (0, 1548)	0.18974588336138443
  (0, 1466)	0.38522283122889006
  (0, 1314)	0.2870732867707909
  (0, 1242)	0.2851658957969986
  (0, 1149)	0.24295165440804062
  (0, 922)	0.33967994947666136
  (0, 588)	0.21360263897569232
  (0, 575)	0.13794931877835728
  (0, 365)	0.34397270094406457
  (0, 104)	0.38522283122889006
  (0, 60)	0.32999325878997476
  (1, 1597)	0.19600173638743415
  (1, 1577)	0.18104132752523375
  (1, 1548)	0.17811624103437498
  (1, 1400)	0.36551974570240714
  (1, 1390)	0.24184930137536473
  (1, 1072)	0.40481324926648843
  (1, 876)	0.19806677987812946
  (1, 783)	0.2792627929850498
  (1, 776)	0.1370599714783049
  (1, 529)	0.29080305025182945
  (1, 517)	0.29338152784833804
  (1, 406)	0.29471050986090247
  (1, 296)	0.3449943378001189
  :	:
  (1707, 1341)	0.22479831571431766
  (1707, 1328)	0.14920869599709383
  (1707, 1322)	0.1183196511286362
  (1707, 1249)	0.1440927717903699
  (1707, 1190)	0.1599055075451517
  (1707, 

In [24]:
recipe_df.difficulty.unique()

array(['Not too tricky', 'Super easy', 'Showing off'], dtype=object)

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nlp import ingredient_parser
import pickle
import unidecode, ast

# Top-N recomendations order by score
def get_recommendations(N, scores):
    # load in recipe dataset 
    df_recipes = pd.read_csv('input_data/parsed_ingreds_recipes.csv')
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations 
    recommendation = pd.DataFrame(columns = ['recipe', 'ingredients', 'score', 'url'])
    count = 0
    for i in top:
        recommendation.at[count, 'recipe'] = title_parser(df_recipes['recipe_name'][i])
        recommendation.at[count, 'ingredients'] = ingredient_parser_final(df_recipes['ingredients'][i])
        recommendation.at[count, 'url'] = df_recipes['recipe_urls'][i]
        recommendation.at[count, 'score'] = "{:.3f}".format(float(scores[i]))
        count += 1
    return recommendation

# neaten the ingredients being outputted 
def ingredient_parser_final(ingredient):
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ast.literal_eval(ingredient)
    
    ingredients = ','.join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients

def title_parser(title):
    title = unidecode.unidecode(title)
    return title 

def RecSys(ingredients, N=5):
    """
    The reccomendation system takes in a list of ingredients and returns a list of top 5 
    recipes based of of cosine similarity. 
    :param ingredients: a list of ingredients
    :param N: the number of reccomendations returned 
    :return: top 5 reccomendations for cooking recipes
    """

    # load in tdidf model and encodings 
    with open('model/tfidf_encoding.pkl', 'rb') as f:
        tfidf_encodings = pickle.load(f)

    with open("model/tfidf_model.pkl", "rb") as f:
        tfidf = pickle.load(f)

    # parse the ingredients using my ingredient_parser 
    try: 
        ingredients_parsed = ingredient_parser(ingredients)
    except:
        ingredients_parsed = ingredient_parser([ingredients])
    
    # use our pretrained tfidf model to encode our input ingredients
    ingredients_tfidf = tfidf.transform([ingredients_parsed])

    # calculate cosine similarity between actual recipe ingreds and test ingreds
    cos_sim = map(lambda x: cosine_similarity(ingredients_tfidf, x), tfidf_encodings)
    scores = list(cos_sim)

    # Filter top N recommendations 
    recommendations = get_recommendations(N, scores)
    return recommendations

if __name__ == "__main__":
    # test ingredients
    test_ingredients = "pasta, tomato, onion"
    recs = RecSys(test_ingredients)
    print(recs.score)


ModuleNotFoundError: No module named 'scipy.sparse._csr'